# Tic Tac Toe

Tic Tac Toe is a famous childrens' game also known as noughts and crosses. The game is played by placing x'es (crosses) and 'o's (noughts) on a 3x3 grid playing field.

The first player to fill a column, row, or diagonal is the winner.

We will implemenent the `MiniMax` algorithm for this game.

## State Representation

I use a simple two-dimensional array as my datastructure. 

### Initial State

The initial state is an empty board.

In [106]:
EMPTY = '_'
NOUGHT = 'o'
CROSS = 'x'

initial = [ [ EMPTY, EMPTY, EMPTY ],
            [ EMPTY, EMPTY, EMPTY ],
            [ EMPTY, EMPTY, EMPTY ]
          ]

## Evaluation Function

Next I implement a function to evaluate if a game state is terminal and if it resulted in
a win, loose, or draw.

Instead of only allowing a 3x3 playing field, I also allow generalizations to any rectangular field.

We define several utility functions to help in the implementation.

In [107]:
WIN = +1
LOOSE = -1
DRAW = 0

def calcEmpty( state ):
    count = 0
    for i in range( len(state) ):
        for j in range( len(state[i])):
            if ( state[i][j] == EMPTY ):
                count = count + 1
    return count

def isComplete( state ):
    return calcEmpty(state) == 0

def isTerminal( state ):
    return ( evaluate( state ) is not None )

def isWin( player, state ):
    width, height = len(state[0]), len( state )
    # Check rows
    for i in range( height ):
        tmp = True
        for j in range( width ):
            if ( state[i][j] != player ):
                tmp = False
                break
        if ( tmp ):
            return True
    
    # Check columns
    for j in range( width ):
        tmp = True
        for i in range( height ):
            if ( state[i][j] != player ):
                tmp = False
                break
        if ( tmp ):
            return True
        
    # Check diagonal
    if ( width >= height ):
        # 3 rows, 5 columns
        limit = height
        for offset in range( width - limit + 1):
            tmp = True
            for d in range( limit ):
                if ( state[d][d+offset] != player ):
                    tmp = False
                    break
            if ( tmp ):
                return True
            tmp = True
            for d in range( limit ):
                if ( state[height - 1 - d][d+offset] != player ):
                    tmp = False
                    break
            if ( tmp ):
                return True
    else:
        # 5 rows, 3 columns
        limit = width
        for offset in range( height - limit + 1):
            tmp = True
            for d in range( limit ):
                if ( state[d+offset][d] != player ):
                    tmp = False
                    break
            if ( tmp ):
                return True
            tmp = True
            for d in range( limit ):
                if ( state[d+offset][width-1-d] != player ):
                    tmp = False
                    break
            if ( tmp ):
                return True
    return False

def evaluate( state ):
    result = None
    if ( isWin( CROSS, state ) ):
        result = WIN
    elif (isWin( NOUGHT, state ) ):
        result = LOOSE
    elif ( isComplete( state ) ):
        result = DRAW
    return result

In [108]:
print( evaluate( [ [ EMPTY, EMPTY, EMPTY ], [ EMPTY, EMPTY, EMPTY ],[ EMPTY, EMPTY, EMPTY ] ] ) )

None


In [109]:
print( evaluate( [ [ CROSS, CROSS, CROSS ], [ EMPTY, EMPTY, EMPTY ],[ EMPTY, EMPTY, EMPTY ] ] ) )

1


In [110]:
print( evaluate( [ [ CROSS, CROSS, NOUGHT ], [ EMPTY, NOUGHT, EMPTY ],[ NOUGHT, EMPTY, EMPTY ] ] ) )

-1


In [111]:
print( evaluate( [ [ CROSS, CROSS, NOUGHT ], [ EMPTY, NOUGHT, NOUGHT ],[ EMPTY, EMPTY, NOUGHT ] ] ) )

-1


In [112]:
def successor( player, state ):
    children = []
    #print('successor', state, 'isTerminal', isTerminal(state) )
    if ( not isTerminal( state ) ):
        for i in range( len(state) ):
            for j in range( len(state[i])):
                if ( state[i][j] == EMPTY ):
                    # print('found empty',i,j)
                    ns = [ [ c for c in r ] for r in state ]
                    ns[i][j] = player
                    children = children + [ ns ]
    return children

In [113]:
print( successor( CROSS, [ [ CROSS, CROSS, NOUGHT ], [ EMPTY, NOUGHT, NOUGHT ],[ EMPTY, EMPTY, NOUGHT ] ])) 

[]


In [114]:
def otherPlayer( player ):
    if ( player == CROSS ):
        other = NOUGHT
    elif ( player == NOUGHT ):
        other = CROSS
    return other
          
def createTree( player, root, maxLevel = None, level = 0 ):
    #print('createTree', root, player)
    tree = [ root, evaluate( root ), None, [] ]
    children = successor( player, root )
    #print('createTree children', children)
    if ( maxLevel is None ) or ( level < maxLevel ):
        for c in children:
            childTree = createTree( otherPlayer( player ), c, maxLevel, level + 1 )
            tree[ 3 ] = tree[ 3 ] + [ childTree ]
    return tree

    
def genMiniMax( player, tree ):
    s, ev, minimax, children = tree
    if ( len(children) == 0 ):
        if ( isTerminal( s ) ):
            tree[2] = ev
        else:
            tree[2] = heuristic( player, s )
    else:
        if ( player == CROSS ):
            factor = 1.0
        elif ( player == NOUGHT ):
            factor = -1.0
        max = None
        for c in children:
            child = genMiniMax( otherPlayer( player ), c )
            if ( max is None ) or ( child[2] * factor > max ):
                max = child[2] * factor
        tree[2] = max * factor
    return tree


In [115]:
def printState( state, level = 0 ):
    for i in range( len( state ) ):
        print( " " * ( level * 2 ), end = "" )
        for j in range( len( state[i] ) ):
            print( state[i][j], end="")
        print()
        
def printTree( tree, level = 0 ):
    state, value, h, children = tree
    print( " " * ( level * 2 ), end = "" )
    print( "Eval:", value, 'MiniMax:', h, 'Heuristic:', heuristic( CROSS, state )  )
    printState( state, level )
    for c in children:
        printTree( c, level + 1 )

In [116]:
state = [ [ CROSS, CROSS, NOUGHT ], [ EMPTY, NOUGHT, NOUGHT ],[ EMPTY, EMPTY, CROSS ] ]

printState( state, 3 )
print( successor( CROSS, state) )

      xxo
      _oo
      __x
[[['x', 'x', 'o'], ['x', 'o', 'o'], ['_', '_', 'x']], [['x', 'x', 'o'], ['_', 'o', 'o'], ['x', '_', 'x']], [['x', 'x', 'o'], ['_', 'o', 'o'], ['_', 'x', 'x']]]


In [117]:
tree = createTree( CROSS, state, 1 )
printTree( tree )
tree = genMiniMax( CROSS, tree )
printTree( tree )

Eval: None MiniMax: None Heuristic: 0
xxo
_oo
__x
  Eval: None MiniMax: None Heuristic: 1
  xxo
  xoo
  __x
  Eval: None MiniMax: None Heuristic: 2
  xxo
  _oo
  x_x
  Eval: None MiniMax: None Heuristic: 1
  xxo
  _oo
  _xx
Eval: None MiniMax: 2.0 Heuristic: 0
xxo
_oo
__x
  Eval: None MiniMax: 1 Heuristic: 1
  xxo
  xoo
  __x
  Eval: None MiniMax: 1 Heuristic: 2
  xxo
  _oo
  x_x
  Eval: None MiniMax: 2 Heuristic: 1
  xxo
  _oo
  _xx


## Number of Nodes and Branching Factor

To help in our further analysis, I want to write a function that calculate some characteristics of our search space.

  * `countNumberOfNodes` : returns the number of nodes in a tree.
     
  * `calcBranchingFactor` : returns the average branching factor in a tree.
     
  * `calcAverageDepth` : returns the average depth of the leaf nodes in a tree.
     
     

In [118]:
def countNumberOfNodes( tree ):
    _,_,_,children = tree
    numChildren = 0
    for c in children:
        numChildren = numChildren + countNumberOfNodes( c )
    return 1 + numChildren

def countNumberOfLeafNodes( tree ):
    _,_,_,children = tree
    if ( len(children) == 0 ):
        numLeaves = 1
    else:
        numLeaves = 0
        for c in children:
            numLeaves = numLeaves + countNumberOfLeafNodes( c )
    return numLeaves

def countNumberOfChildren( tree ):
    _,_,_,children = tree
    numChildren = 0
    for c in children:
        numChildren = numChildren + countNumberOfNodes( c )
    return numChildren

def calcAverageBranchingFactor( tree ):
    return countNumberOfChildren( tree ) / countNumberOfNodes( tree )
    
def calcDepthCount( tree, level = 0 ):
    _,_,_,children = tree
    if ( len(children) == 0 ):
        depthCount = level
    else:
        depthCount = 0
        for c in children:
            depthCount = depthCount + calcDepthCount( c, level + 1 )
    return depthCount
    
def calcAverageDepthOfLeafNodes( tree ):
    return calcDepthCount( tree, 0 ) / countNumberOfLeafNodes( tree )


In [119]:
numNodes = countNumberOfNodes( tree )
numLeafNodes = countNumberOfLeafNodes( tree )

b = calcAverageBranchingFactor( tree )

print('Number of Nodes', numNodes, 'Number of leaf nodes', numLeafNodes, 'b', b, 'Average depth of leaf nodes', calcAverageDepthOfLeafNodes( tree ) )

Number of Nodes 4 Number of leaf nodes 3 b 0.75 Average depth of leaf nodes 1.0


## Analysis of the Complete Search Space

Now let us analyse the characteristics of the entire Tic Tac Toe search space.

In [120]:
def heuristic( player, state ):
    winCount = 0
    for i in range( len( state ) ):
        for j in range( len( state[0] ) ):
            if ( state[i][j] == EMPTY ):
                ns = [ [ x for x in row ] for row in state ]
                ns[i][j] = player
                if ( isWin(player, ns ) ):
                    winCount = winCount + 1
    return winCount

In [121]:
state2 = [ [ CROSS, EMPTY, EMPTY ], [ EMPTY, NOUGHT, EMPTY ], [ NOUGHT, EMPTY, CROSS ] ]
tree = createTree( CROSS, state2 )
tree = genMiniMax( CROSS, tree )

print('Tree for state', state2 )
printTree( tree )

numNodes = countNumberOfNodes( tree )
numLeafNodes = countNumberOfLeafNodes( tree )

b = calcAverageBranchingFactor( tree )

print('Number of Nodes', numNodes, 'Number of leaf nodes', numLeafNodes, 'b', b, 'Average depth of leaf nodes', calcAverageDepthOfLeafNodes( tree ) )


Tree for state [['x', '_', '_'], ['_', 'o', '_'], ['o', '_', 'x']]
Eval: None MiniMax: 1.0 Heuristic: 0
x__
_o_
o_x
  Eval: None MiniMax: -1.0 Heuristic: 1
  xx_
  _o_
  o_x
    Eval: -1 MiniMax: -1 Heuristic: 0
    xxo
    _o_
    o_x
    Eval: None MiniMax: 1.0 Heuristic: 1
    xx_
    oo_
    o_x
      Eval: 1 MiniMax: 1 Heuristic: 2
      xxx
      oo_
      o_x
      Eval: None MiniMax: -1.0 Heuristic: 1
      xx_
      oox
      o_x
        Eval: -1 MiniMax: -1 Heuristic: 0
        xxo
        oox
        o_x
        Eval: None MiniMax: 1.0 Heuristic: 1
        xx_
        oox
        oox
          Eval: 1 MiniMax: 1 Heuristic: 0
          xxx
          oox
          oox
      Eval: None MiniMax: -1.0 Heuristic: 1
      xx_
      oo_
      oxx
        Eval: -1 MiniMax: -1 Heuristic: 0
        xxo
        oo_
        oxx
        Eval: -1 MiniMax: -1 Heuristic: 1
        xx_
        ooo
        oxx
    Eval: None MiniMax: 1.0 Heuristic: 1
    xx_
    _oo
    o_x
      Eval: 1 MiniM

        x_o
        oox
        oxx
    Eval: None MiniMax: 1.0 Heuristic: 1
    x__
    _ox
    oox
      Eval: None MiniMax: -1.0 Heuristic: 1
      xx_
      _ox
      oox
        Eval: -1 MiniMax: -1 Heuristic: 0
        xxo
        _ox
        oox
        Eval: None MiniMax: 1.0 Heuristic: 1
        xx_
        oox
        oox
          Eval: 1 MiniMax: 1 Heuristic: 0
          xxx
          oox
          oox
      Eval: 1 MiniMax: 1 Heuristic: 2
      x_x
      _ox
      oox
      Eval: None MiniMax: -1.0 Heuristic: 1
      x__
      xox
      oox
        Eval: -1 MiniMax: -1 Heuristic: 1
        xo_
        xox
        oox
        Eval: -1 MiniMax: -1 Heuristic: 0
        x_o
        xox
        oox
  Eval: None MiniMax: -1.0 Heuristic: 0
  x__
  _o_
  oxx
    Eval: None MiniMax: 0.0 Heuristic: 0
    xo_
    _o_
    oxx
      Eval: None MiniMax: 0.0 Heuristic: 1
      xox
      _o_
      oxx
        Eval: None MiniMax: 1.0 Heuristic: 1
        xox
        oo_
        oxx
       

In [130]:
tree = genMiniMax( CROSS, createTree( CROSS, initial ) ) 
print( 'Tic Tac Toe evaluation MiniMax', tree[2] )

Tic Tac Toe evaluation MiniMax 0.0


In [131]:
numNodes = countNumberOfNodes( tree )
numLeafNodes = countNumberOfLeafNodes( tree )

b = calcAverageBranchingFactor( tree )

print('Number of Nodes', numNodes, 'Number of leaf nodes', numLeafNodes, 'b', b, 'Average depth of leaf nodes', calcAverageDepthOfLeafNodes( tree ) )

Number of Nodes 549946 Number of leaf nodes 255168 b 0.9999981816396519 Average depth of leaf nodes 8.25451467268623


In [124]:
print(countNumberOfNodes( tree ) )

18730


In [125]:
def destroy( args ):
    if ( args < 2 ):
        result = 1
    else: 
        result = destroy( args - 1 ) + destroy( args - 2 )
    return result

In [126]:
destroy(10)

89

In [127]:
for i in range(20):
    print('i',i,destroy(i) )
    

i 0 1
i 1 1
i 2 2
i 3 3
i 4 5
i 5 8
i 6 13
i 7 21
i 8 34
i 9 55
i 10 89
i 11 144
i 12 233
i 13 377
i 14 610
i 15 987
i 16 1597
i 17 2584
i 18 4181
i 19 6765


In [128]:
def fib( args ):
    dptable = [ 1, 1 ]
    if ( args < 2 ):
        result = dptable[args]
    else:
        for i in range( 2, args + 1 ):
            dptable = dptable + [ dptable[i - 1] + dptable[i - 2 ] ]
        result = dptable[args]
    return result


In [129]:
for i in range(20):
    print('i',i,fib(i) )

i 0 1
i 1 1
i 2 2
i 3 3
i 4 5
i 5 8
i 6 13
i 7 21
i 8 34
i 9 55
i 10 89
i 11 144
i 12 233
i 13 377
i 14 610
i 15 987
i 16 1597
i 17 2584
i 18 4181
i 19 6765
